In [161]:
import numpy as np
import pandas as pd
import csv
import json
from sklearn import preprocessing

In [162]:
'''
ID - an Id that represents a (Shop, Item) tuple within the test set
shop_id - unique identifier of a shop
item_id - unique identifier of a product
item_category_id - unique identifier of item category
item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
item_price - current price of an item
date - date in format dd/mm/yyyy
date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
item_name - name of item
shop_name - name of shop
item_category_name - name of item category
'''

'\nID - an Id that represents a (Shop, Item) tuple within the test set\nshop_id - unique identifier of a shop\nitem_id - unique identifier of a product\nitem_category_id - unique identifier of item category\nitem_cnt_day - number of products sold. You are predicting a monthly amount of this measure\nitem_price - current price of an item\ndate - date in format dd/mm/yyyy\ndate_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33\nitem_name - name of item\nshop_name - name of shop\nitem_category_name - name of item category\n'

In [163]:
items = pd.read_csv("../data/items.csv")
item_cat = pd.read_csv("../data/item_categories.csv")
train = pd.read_csv("../data/sales_train.csv")
sample_submission = pd.read_csv("../data/sample_submission.csv")
shops = pd.read_csv("../data/shops.csv")
test = pd.read_csv("../data/test.csv")

In [164]:
print('items : ', len(items))
print('item_cat : ', len(item_cat))
print('train : ' , len(train))
print('test : ', len(test))
print('shops : ', len(shops))

items :  22170
item_cat :  84
train :  2935849
test :  214200
shops :  60


In [165]:
print(min(train['item_cnt_day']), max(train['item_cnt_day']))

-22.0 2169.0


In [166]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [167]:
item_cat.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [168]:
train.head(10)
# train.tail()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
5,10.01.2013,0,25,2564,349.00,1.0
6,02.01.2013,0,25,2565,549.00,1.0
7,04.01.2013,0,25,2572,239.00,1.0
8,11.01.2013,0,25,2572,299.00,1.0
9,03.01.2013,0,25,2573,299.00,3.0


In [169]:
train['date'] = train['date'].str[3:]

In [170]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,01.2013,0,59,22154,999.00,1.0
1,01.2013,0,25,2552,899.00,1.0
2,01.2013,0,25,2552,899.00,-1.0
3,01.2013,0,25,2554,1709.05,1.0
4,01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.2015,33,25,7409,299.00,1.0
2935845,10.2015,33,25,7460,299.00,1.0
2935846,10.2015,33,25,7459,349.00,1.0
2935847,10.2015,33,25,7440,299.00,1.0


In [171]:
train_groupby = train.groupby(['shop_id', 'item_id', 'date'])['item_cnt_day'].agg('sum').reset_index()
train_groupby

,shop_id,item_id,date,item_cnt_day
0,0,30,02.2013,31.0
1,0,31,02.2013,11.0
2,0,32,01.2013,6.0
3,0,32,02.2013,10.0
4,0,33,01.2013,3.0
...,...,...,...,...
1609119,59,22164,04.2015,2.0
1609120,59,22164,07.2015,1.0
1609121,59,22167,06.2014,1.0
1609122,59,22167,10.2013,1.0


In [172]:
train_groupby_item[train_groupby_item['item_id'] == 2552]

,item_id,item_cnt_day
2495,2552,5.0


In [173]:
train_groupby_shop = train_groupby.groupby(['shop_id'])['item_cnt_day'].agg('sum').reset_index()
train_groupby_item = train_groupby.groupby(['item_id'])['item_cnt_day'].agg('sum').reset_index()
train_groupby_shop_and_item = train_groupby.groupby(['shop_id', 'item_id','date'])['item_cnt_day'].agg('sum').reset_index()

In [174]:
sample_submission.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [175]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [176]:
test.head() ##언젠지 없냐?

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [177]:
min(items['item_category_id'])

0

In [178]:
train_groupby_shop.head()

,shop_id,item_cnt_day
0,0,11705.0
1,1,6311.0
2,2,30620.0
3,3,28355.0
4,4,43942.0


In [179]:
train_groupby_shop = train_groupby_shop.rename(columns = {'item_cnt_day': '#2_amount'}, inplace = False)
train = pd.merge(train, train_groupby_shop, on='shop_id')

In [180]:
train['tmp'] = train['item_price']*train['item_cnt_day']
train_groupby_shop_price = train.groupby(['shop_id'])['tmp'].agg('sum').reset_index()
train_groupby_shop_price = train_groupby_shop_price.rename(columns = {'tmp': '#2_price'}, inplace = False)
train = pd.merge(train, train_groupby_shop_price, on ='shop_id')

In [181]:
train = pd.merge(train, items[['item_id', 'item_category_id']], on = 'item_id')

In [182]:
train_groupby_item_category = train.groupby(['item_category_id'])['item_cnt_day'].agg('sum').reset_index()

In [184]:
train = pd.merge(train, train_groupby_item_category , on='item_category_id')

In [185]:
train = train.rename(columns = {'item_cnt_day_y': '#3_amount'}, inplace = False)

In [186]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day_x,#2_amount,tmp,#2_price,item_category_id,#3_amount
0,01.2013,0,59,22154,999.0,1.0,48993.0,999.0,4.522658e+07,37,203284.0
1,01.2013,0,25,22154,999.0,1.0,241920.0,999.0,2.164806e+08,37,203284.0
2,01.2013,0,25,22154,999.0,1.0,241920.0,999.0,2.164806e+08,37,203284.0
3,01.2013,0,25,22154,999.0,1.0,241920.0,999.0,2.164806e+08,37,203284.0
4,01.2013,0,25,22154,999.0,1.0,241920.0,999.0,2.164806e+08,37,203284.0
...,...,...,...,...,...,...,...,...,...,...,...
2935844,10.2015,33,55,13093,250.0,1.0,63388.0,250.0,4.979206e+07,36,22.0
2935845,09.2015,32,55,13091,1000.0,1.0,63388.0,1000.0,4.979206e+07,36,22.0
2935846,09.2015,32,55,13094,2500.0,1.0,63388.0,2500.0,4.979206e+07,36,22.0
2935847,09.2015,32,55,13094,2500.0,2.0,63388.0,5000.0,4.979206e+07,36,22.0


In [187]:
train_groupby_item = train_groupby_item.rename(columns = {'item_cnt_day' : '#4_amount'}, inplace = False)

In [188]:
train = pd.merge(train, train_groupby_item , on='item_id')

In [189]:
train = train.drop(['item_cnt_day_x'], axis = 1)

In [190]:
train['month'] = train['date'].str[:2]

In [191]:
train['year'] = train['date'].str[3:]

In [192]:
def normalize_column(df, column_name):
    x = df[[column_name]]
    # Create x, where x the 'scores' column's values as floats
    x = x.values.astype(float)
    # Create a minimum and maximum processor object
    min_max_scaler = preprocessing.MinMaxScaler()
    # Create an object to transform the data to fit minmax processor
    x_scaled = min_max_scaler.fit_transform(x)
    # Run the normalizer on the dataframe
    df_normalized = pd.DataFrame(x_scaled, columns = [column_name])
    
    df[column_name] = df_normalized
    
    return df

In [193]:
normalize_column(train, '#2_amount')
normalize_column(train, '#2_price')
normalize_column(train, '#3_amount')
normalize_column(train, '#4_amount')
train['month'] = train['month'].astype(int)
train['year'] = train['year'].astype(int)

In [195]:
train.drop('tmp', axis = 1)

,date,date_block_num,shop_id,item_id,item_price,#2_amount,#2_price,item_category_id,#3_amount,#4_amount,month,year
0,01.2013,0,59,22154,999.0,0.156751,0.190977,37,0.320550,0.000373,1,2013
1,01.2013,0,25,22154,999.0,0.778200,0.920216,37,0.320550,0.000373,1,2013
2,01.2013,0,25,22154,999.0,0.778200,0.920216,37,0.320550,0.000373,1,2013
3,01.2013,0,25,22154,999.0,0.778200,0.920216,37,0.320550,0.000373,1,2013
4,01.2013,0,25,22154,999.0,0.778200,0.920216,37,0.320550,0.000373,1,2013
...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,10.2015,33,55,13093,250.0,0.203120,0.210418,36,0.000033,0.000123,10,2015
2935845,09.2015,32,55,13091,1000.0,0.203120,0.210418,36,0.000033,0.000064,9,2015
2935846,09.2015,32,55,13094,2500.0,0.203120,0.210418,36,0.000033,0.000075,9,2015
2935847,09.2015,32,55,13094,2500.0,0.203120,0.210418,36,0.000033,0.000075,9,2015
